In [1]:
# 智慧工厂设备7天内故障预测：A题建模与可视化分析
# 2025“创新杯”大数据挑战赛

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    precision_score, recall_score, roc_curve, auc, precision_recall_curve
)
from sklearn.ensemble import VotingClassifier, StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
import optuna
import shap
import warnings
warnings.filterwarnings('ignore')

# =========================== 1. 数据读取与初步清洗 ===========================
file_path = r'D:\桌面\2025年第四届“创新杯”（原钉钉杯）大学生大数据挑战赛初赛题目\2025年第四届“创新杯”（原钉钉杯）大学生大数据挑战赛初赛题目\A题\data\train_data.csv'
df = pd.read_csv(file_path)
df.groupby('Failure_Within_7_Days')['Operational_Hours'].describe()


d:\Anaconda\envs\dl2025-py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,count,mean,std,min,25%,50%,75%,max
Failure_Within_7_Days,,,,,,,,
False,375838.0,47123.514743,27313.363499,0.0,23502.0,47012.0,70499.0,100000.0
True,24162.0,95305.478189,3610.151596,72361.0,93144.0,96088.0,98227.0,100000.0


In [ ]:

# =========================== 2. 可视化增强：缺失值图、字段分布、特征与标签关系图 ===========================
missing_ratio = df.isnull().mean().sort_values(ascending=False)
plt.figure(figsize=(10,6))
missing_ratio.plot(kind='bar', color='salmon')
plt.title("Missing Value Ratio by Feature")
plt.ylabel("Missing Ratio")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("missing_ratio_bar.png")
plt.close()

numeric_features = df.select_dtypes(include=np.number).columns.tolist()
for col in numeric_features[:10]:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True, bins=30, color='steelblue')
    plt.title(f"Distribution: {col}")
    plt.tight_layout()
    plt.savefig(f"dist_{col}.png")
    plt.close()

for col in numeric_features[:6]:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df['Failure_Within_7_Days'], y=df[col])
    plt.title(f"{col} vs Failure")
    plt.tight_layout()
    plt.savefig(f"box_{col}.png")
    plt.close()

# =========================== 3. 特征清洗与转换 ===========================
unused_cols = ['Laser_Intensity','Hydraulic_Pressure_bar','Coolant_Flow_L_min','Heat_Index']
df.drop(columns=unused_cols, inplace=True)

for col in ['AI_Supervision','Failure_Within_7_Days']:
    df[col] = df[col].astype(int)

df['Machine_Type'] = LabelEncoder().fit_transform(df['Machine_Type'])

for col in df.columns[df.isnull().any()]:
    df[col] = df[col].fillna(df[col].median())

# =========================== 4. 异常值处理 ===========================
def clip_iqr(data, features):
    for f in features:
        q1, q3 = data[f].quantile([0.25,0.75])
        iqr = q3 - q1
        data[f] = data[f].clip(q1 - 1.5*iqr, q3 + 1.5*iqr)
    return data

plt.figure(figsize=(8,4))
sns.boxplot(x=df['Temperature_C'])
plt.title('Temp Before IQR')
plt.tight_layout()
plt.savefig('temp_before_iqr.png')
plt.close()

num_feats = df.select_dtypes(include=np.number).columns.drop(['Failure_Within_7_Days','Remaining_Useful_Life_days'])
df = clip_iqr(df, num_feats)

plt.figure(figsize=(8,4))
sns.boxplot(x=df['Temperature_C'])
plt.title('Temp After IQR')
plt.tight_layout()
plt.savefig('temp_after_iqr.png')
plt.close()

# =========================== 5. 特征工程 ===========================
df['Avg_Maintenance_Interval'] = df['Operational_Hours']/(df['Maintenance_History_Count']+1)
df['Historical_Failure_Rate'] = df['Failure_History_Count']/(df['Operational_Hours']+1)
df['Error_Code_Frequency'] = df['Error_Codes_Last_30_Days']/30
df['Maintenance_Frequency'] = df['Maintenance_History_Count']/(df['Last_Maintenance_Days_Ago']+1)

# =========================== 6. 数据划分与标准化 ===========================
X = df.drop(columns=['Machine_ID','Remaining_Useful_Life_days','Failure_Within_7_Days'])
y = df['Failure_Within_7_Days']
X_temp,X_test,y_temp,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_temp,y_temp,test_size=0.2,stratify=y_temp,random_state=42)

scaler = StandardScaler()
X_train[num_feats] = scaler.fit_transform(X_train[num_feats])
X_val[num_feats] = scaler.transform(X_val[num_feats])
X_test[num_feats] = scaler.transform(X_test[num_feats])

# =========================== 7. 特征选择 ===========================
sel = LGBMClassifier(random_state=42)
sel.fit(X_train, y_train)
selector = SelectFromModel(sel, threshold='median', prefit=True)
X_train_sel = selector.transform(X_train)
X_val_sel = selector.transform(X_val)
X_test_sel = selector.transform(X_test)

imp = sel.feature_importances_
feat_names = X_train.columns
selected_names = feat_names[selector.get_support()]
imp_values = imp[selector.get_support()]
plt.figure(figsize=(10,6))
sns.barplot(x=imp_values, y=selected_names)
plt.title('Top Feature Importances')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

# =========================== 8. 处理不平衡 ===========================
plt.figure(figsize=(6,4))
y_train.value_counts().plot(kind='bar')
plt.title('Train Label Before SMOTE')
plt.tight_layout()
plt.savefig('label_before_smote.png')
plt.close()

smote = SMOTE(random_state=42)
X_train_res,y_train_res = smote.fit_resample(X_train_sel, y_train)

plt.figure(figsize=(6,4))
pd.Series(y_train_res).value_counts().plot(kind='bar',color=['green','red'])
plt.title('Train Label After SMOTE')
plt.tight_layout()
plt.savefig('label_after_smote.png')
plt.close()

# =========================== 9. 定义Optuna搜索空间与调参目标函数 ===========================

def xgb_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5),
        "use_label_encoder": False,
        "eval_metric": "logloss",
        "random_state": 42
    }

def lgb_space(trial):
    return {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 30),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5),
        "random_state": 42
    }

def cat_space(trial):
    return {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "random_state": 42,
        "verbose": False
    }

def objective(trial, model_cls, space_fn, Xd, yd):
    params = space_fn(trial)
    model = model_cls(**params)
    score = cross_val_score(model, Xd, yd, cv=3, scoring='roc_auc').mean()
    return score

def tune_model(model_cls, space_fn, Xd, yd):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda t: cross_val_score(model_cls(**space_fn(t)), Xd, yd, cv=3, scoring='roc_auc').mean(), n_trials=50)
    
    # 尝试生成交互式图并保存为 HTML（无需 Kaleido）
    try:
        fig = optuna.visualization.plot_optimization_history(study)
        fig.write_html("optuna_auc_history.html")
        print("已保存调参优化历史图：optuna_auc_history.html")
    except Exception as e:
        print(f"[警告] 无法保存优化历史图：{e}")
    
    print(f"Best params for {model_cls.__name__}: {study.best_params}")
    print(f"Best AUC: {study.best_value:.4f}")
    
    return study.best_params


# =========================== 10. 调参并训练模型 ===========================
xgb_best = tune_model(XGBClassifier, xgb_space, X_train_res, y_train_res)
model_xgb = XGBClassifier(**xgb_best).fit(X_train_res, y_train_res)

lgb_best = tune_model(LGBMClassifier, lgb_space, X_train_res, y_train_res)
model_lgb = LGBMClassifier(**lgb_best).fit(X_train_res, y_train_res)

cat_best = tune_model(CatBoostClassifier, cat_space, X_train_res, y_train_res)
model_cat = CatBoostClassifier(**cat_best).fit(X_train_res, y_train_res)

# =========================== 11. 融合策略 ===========================
voting = VotingClassifier([('xgb', model_xgb), ('lgb', model_lgb), ('cat', model_cat)], voting='soft')
voting.fit(X_train_res, y_train_res)

def blend_proba(Xd):
    return np.mean([m.predict_proba(Xd)[:,1] for m in [model_xgb, model_lgb, model_cat]], axis=0)

stack = StackingClassifier(
    estimators=[('xgb', model_xgb), ('lgb', model_lgb), ('cat', model_cat)],
    final_estimator=LGBMClassifier(n_estimators=100, random_state=42),
    cv=3,
    passthrough=True
)
stack.fit(X_train_res, y_train_res)

# =========================== 12. 模型评估与对比 ===========================
from sklearn.metrics import f1_score

def evaluate(name, model, Xv, yv, compare_list):
    probs = blend_proba(Xv) if name == 'Blending' else model.predict_proba(Xv)[:,1]
    preds = (probs >= 0.5).astype(int)
    fpr, tpr, _ = roc_curve(yv, probs)
    roc_auc = auc(fpr, tpr)
    prec, rec, _ = precision_recall_curve(yv, probs)
    f1 = f1_score(yv, preds)
    compare_list.append({
        "Model": name,
        "Accuracy": (preds == yv).mean(),
        "Precision": precision_score(yv, preds),
        "Recall": recall_score(yv, preds),
        "F1": f1,
        "AUC": roc_auc
    })
    plt.figure()
    plt.plot(fpr, tpr, label=f'AUC={roc_auc:.2f}')
    plt.title(f'{name} ROC Curve')
    plt.legend()
    plt.savefig(f'{name}_roc.png')
    plt.close()
    plt.figure()
    plt.plot(rec, prec)
    plt.title(f'{name} PR Curve')
    plt.savefig(f'{name}_pr.png')
    plt.close()
    print(f"{name} evaluation done.")

compare_results = []
for nm, mdl in [('XGB', model_xgb), ('LGB', model_lgb), ('Cat', model_cat), ('Voting', voting), ('Stacking', stack), ('Blending', None)]:
    if nm == 'Blending':
        evaluate(nm, None, X_val_sel, y_val, compare_results)
    else:
        evaluate(nm, mdl, X_val_sel, y_val, compare_results)

compare_df = pd.DataFrame(compare_results)
compare_df.to_csv("model_comparison.csv", index=False)

# =========================== 13. SHAP 解释增强 ===========================
X_meta = stack.transform(X_test_sel)
meta = stack.final_estimator_
expl = shap.TreeExplainer(meta)
shap_vals = expl.shap_values(X_meta)

shap.summary_plot(shap_vals, X_meta, show=False)
plt.savefig('shap_summary.png')
plt.close()

# 解释单个预测概率最高样本
sample_idx = np.argmax(meta.predict_proba(X_meta)[:,1])
sample_data = X_meta[sample_idx:sample_idx+1]
shap.initjs()
shap.force_plot(expl.expected_value, shap_vals[sample_idx], sample_data, matplotlib=True, show=False)
plt.savefig("shap_force_sample.png")
plt.close()

# 结束，所有图和结果均已保存
